In [1]:
from os.path import expanduser, basename
import warnings
from logging import getLogger
import numpy as np
import pandas as pd
import rasters as rt
import colored_logging as cl
from ECOSTRESS import open_granule
from ECOSTRESS.ECOSTRESS_gridded_tiled_granule import ECOSTRESSTiledGranule
from ECOSTRESS.calval.tower import Tower
from ECOSTRESS.calval.generate_3x3_subset_geometry import generate_3x3_subset_geometry
from ECOSTRESS.calval.extract_tower_calval_from_C2_granule import extract_tower_calval_from_C2_granule
from ECOSTRESS.calval.extract_tower_calval_from_C1_granule import extract_tower_calval_from_C1_granule

In [2]:
logger = getLogger(__name__)

In [3]:
C2_granule_filename = expanduser("~/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip")
tower_list_filename = "ECOSTRESS/calval/2024-08-02_tower_short_list.csv"

In [4]:
C2_granule = open_granule(C2_granule_filename)
C2_granule

[2024-08-16 14:43:03 INFO] loading Collection 2 L3T ET ALEXI: /Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip


L3TETALEXI("/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip")

In [5]:
C2_granule.filename

'/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip'

In [6]:
tile = C2_granule.tile
tile

'16RGV'

In [7]:
tower_list = pd.read_csv(tower_list_filename)
tower_list_at_tile = tower_list[tower_list.tile == tile]
tower_list_at_tile

,tower,name,tile,latitude,longitude
33,US-xJE,NEON Jones Ecological Research Center (JERC),16RGV,31.1948,-84.4686


In [8]:
tower_locations_row = tower_list_at_tile.iloc[0]
tower_locations_row

tower                                              US-xJE
name         NEON Jones Ecological Research Center (JERC)
tile                                                16RGV
latitude                                          31.1948
longitude                                        -84.4686
Name: 33, dtype: object

In [9]:
tower = tower_locations_row.tower
lat = tower_locations_row.latitude
lon = tower_locations_row.longitude
tower_point_latlon = rt.Point(lon, lat)
tower_point_latlon

POINT (-84.4686 31.1948)

In [10]:
C2_granule.geometry.intersects(tower_point_latlon.to_crs(C2_granule.geometry.crs))

True

In [11]:
tower_row, tower_col = C2_granule.geometry.index_point(tower_point_latlon.to_crs(C2_granule.geometry.crs))
rows, cols = C2_granule.geometry.shape
subset_3x3 = C2_granule.geometry[max(tower_row - 1, 0):min(tower_row + 2, rows - 1),
                max(tower_col - 1, 0):min(tower_col + 2, cols - 1)]
subset_3x3


{
  "dimensions": {
    "rows": 3,
    "cols": 3
  },
  "bbox": {
    "xmin": 741120.0,
    "ymin": 3453840.0,
    "xmax": 741330.0,
    "ymax": 3454050.0
  },
  "crs": "EPSG:32616",
  "resolution": {
    "cell_width": 70.0,
    "cell_height": -70.0
  }
}

In [12]:
tower = Tower(tower, lat, lon)
tower

Tower(name="US-xJE", lat=31.1948, lon=-84.4686)

In [19]:
C2_df = extract_tower_calval_from_C2_granule(C2_granule, tower)
C2_df

[2024-08-16 14:55:41 INFO] started reading L3T_ET_ALEXI ETdaily: zip:///Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip!/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01_ETdaily.tif
[2024-08-16 14:55:41 INFO] projecting L3T_ET_ALEXI ETdaily
[2024-08-16 14:55:41 INFO] finished reading L3T_ET_ALEXI ETdaily (0.03s)
[2024-08-16 14:55:41 INFO] Tower(name="US-xJE", lat=31.1948, lon=-84.4686) 2019-08-29 17:08:25 ETdaily: 4.9057536125183105
[2024-08-16 14:55:41 INFO] started reading L3T_ET_ALEXI ETdailyUncertainty: zip:///Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01.zip!/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01/ECOv002_L3T_ET_ALEXI_06509_008_16RGV_20190829T170825_0700_01_ETdailyUncertainty.tif
[2024-08-16 14:55:41 INFO] projecting L3T_ET_ALEXI ETdailyUncertainty
[2024-08-16 14:55:41 INFO] finished reading L3T_ET_ALEXI ET

,tower,orbit,scene,tile,time_UTC,product,variable,med3x3,collection,FieldOfViewObstruction,OrbitCorrectionPerformed,QAPercentCloudCover,QAPercentGoodQuality,filename,lat,lon
0,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdaily,4.905754,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686
1,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdailyUncertainty,0.054000,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686


In [14]:
C1_filename = expanduser("~/Downloads/ALEXI_C1_example/ECOSTRESS_L3_ET_ALEXI_06509_008_20190829T170825_0601_02.h5")

In [15]:
C1_granule = open_granule(C1_filename)
C1_granule

[2024-08-16 14:43:04 INFO] loading Collection 1 L3 ET ALEXI: /Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L3_ET_ALEXI_06509_008_20190829T170825_0601_02.h5
[2024-08-16 14:43:04 INFO] searching pattern: /Users/halverso/Downloads/ALEXI_C1_example/ECO*_L1B_GEO_06509_008_*_*_*.h5
[2024-08-16 14:43:04 INFO] found: /Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L1B_GEO_06509_008_20190829T170825_0601_02.h5
[2024-08-16 14:43:04 INFO] searching pattern: /Users/halverso/Downloads/ALEXI_C1_example/ECO*_L2_CLOUD_06509_008_*_*_*.h5
[2024-08-16 14:43:04 INFO] found: /Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L2_CLOUD_06509_008_20190829T170825_0601_02.h5


L3 disALEXI Granule
{
  "L3 disALEXI": "/Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L3_ET_ALEXI_06509_008_20190829T170825_0601_02.h5",
  "L2 CLOUD:": "/Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L2_CLOUD_06509_008_20190829T170825_0601_02.h5",
  "L1B GEO:": "/Users/halverso/Downloads/ALEXI_C1_example/ECOSTRESS_L1B_GEO_06509_008_20190829T170825_0601_02.h5"
}

In [22]:
C1_df = extract_tower_calval_from_C1_granule(C1_granule, tower)
C1_df

[2024-08-16 14:56:23 INFO] dataset EVAPOTRANSPIRATION ALEXI/ETdaily read with dtype float32
[2024-08-16 14:56:27 INFO] Tower(name="US-xJE", lat=31.1948, lon=-84.4686) 2019-08-29 17:08:25 ETdaily: 5.34009313583374
[2024-08-16 14:56:27 INFO] dataset EVAPOTRANSPIRATION ALEXI/ETdailyUncertainty read with dtype float32
[2024-08-16 14:56:31 INFO] Tower(name="US-xJE", lat=31.1948, lon=-84.4686) 2019-08-29 17:08:25 ETdailyUncertainty: 5.13399076461792
[2024-08-16 14:56:31 INFO] dataset EVAPOTRANSPIRATION ALEXI/QualityFlag read with dtype uint16
[2024-08-16 14:56:36 INFO] Tower(name="US-xJE", lat=31.1948, lon=-84.4686) 2019-08-29 17:08:25 QualityFlag: 0.0


,tower,orbit,scene,tile,time_UTC,product,variable,med3x3,collection,FieldOfViewObstruction,OrbitCorrectionPerformed,QAPercentCloudCover,QAPercentGoodQuality,filename,lat,lon
0,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,ETdaily,5.340093,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686
1,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,ETdailyUncertainty,5.133991,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686
2,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,QualityFlag,0.000000,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686


In [18]:
blank_df = pd.DataFrame([], columns=[
        "tower",
        "orbit",
        "scene",
        "tile",
        "time_UTC",
        "product",
        "variable",
        "med3x3",
        "collection",
        "FieldOfViewObstruction",
        "OrbitCorrectionPerformed",
        "QAPercentCloudCover",
        "QAPercentGoodQuality",
        "filename",
        "lat",
        "lon"
    ])

blank_df

,tower,orbit,scene,tile,time_UTC,product,variable,med3x3,collection,FieldOfViewObstruction,OrbitCorrectionPerformed,QAPercentCloudCover,QAPercentGoodQuality,filename,lat,lon


In [21]:
calval_df = pd.concat([blank_df, C2_df])
calval_df

/var/folders/hk/35_x93nx79vg1zmd2fy1vcbr0000gq/T/ipykernel_82889/2990690320.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  calval_df = pd.concat([blank_df, C2_df])


,tower,orbit,scene,tile,time_UTC,product,variable,med3x3,collection,FieldOfViewObstruction,OrbitCorrectionPerformed,QAPercentCloudCover,QAPercentGoodQuality,filename,lat,lon
0,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdaily,4.905754,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686
1,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdailyUncertainty,0.054000,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686


In [23]:
calval_df = pd.concat([calval_df, C1_df])
calval_df

,tower,orbit,scene,tile,time_UTC,product,variable,med3x3,collection,FieldOfViewObstruction,OrbitCorrectionPerformed,QAPercentCloudCover,QAPercentGoodQuality,filename,lat,lon
0,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdaily,4.905754,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686
1,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,16RGV,2019-08-29 17:08:25,L3T_ET_ALEXI,ETdailyUncertainty,0.054000,2,unknown,True,0.0,98.315236,/Users/halverso/Downloads/ECOv002_L3T_ET_ALEXI...,31.1948,-84.4686
0,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,ETdaily,5.340093,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686
1,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,ETdailyUncertainty,5.133991,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686
2,"Tower(name=""US-xJE"", lat=31.1948, lon=-84.4686)",6509,8,,2019-08-29 17:08:25,L3_ET_ALEXI,QualityFlag,0.000000,1,,,,,/Users/halverso/Downloads/ALEXI_C1_example/ECO...,31.1948,-84.4686


In [24]:
calval_df.to_csv("calval_preview.csv")